In [1]:
import os
import pandas as pd
import numpy as np

os.chdir('../DrugCell')

# Original Implementation

In [2]:
ls ./data/

cell2ind.txt*          drugcell_train.txt     rcellminer_test.txt
cell2mutation.txt      drugcell_val.txt       sanger_cell_lines.txt
drug2fingerprint.txt   gene2ind.txt*          smiles_by_nsc.csv
drug2ind.txt*          nci60Act.csv           test.csv
drugcell_ont.txt       out.txt                test.txt
drugcell_test.txt      out_.txt


In [3]:
pd.read_table('data/rcellminer_test.txt')

,MCF7_BREAST,CC1=CC(=O)C=CC1=O
0,MCF7_BREAST,CCCCCCCCCCCCCCCc1cc(O)ccc1N
1,MCF7_BREAST,OC(=O)CCCc1ccccc1
2,MCF7_BREAST,CCN(CC)CCCNc1c2ccc(Cl)cc2nc3ccc(OC)cc13
3,MCF7_BREAST,CCCCCN(CCCCC)CCCNc1c2CCCCc2nc3ccc(Cl)cc13
4,MCF7_BREAST,[R][Hg]OC(=O)C.c1ccc2c(c1)ccc3ccccc23
...,...,...
981984,CAKI1_KIDNEY,NC(=N)NCCCC1N[R]C(=O)C(CC(=O)O)N[R][R]C1=O
981985,CAKI1_KIDNEY,OC(=O)CC1N[R][R][R]C(=O)C2CCCN2C1=O
981986,CAKI1_KIDNEY,OC(=O)CCC1NC(=O)C(CC(=O)O)N[R][R][R]C1=O
981987,CAKI1_KIDNEY,NC(=O)C1N[R][R]C(=O)CSCC2CCCN2[R]C1=O


In [4]:
!rm -rf Result_sample && mkdir Result_sample 
!rm -rf Hidden_sample/ && mkdir Hidden_sample

!python code/predict_drugcell_cpu.py \
            -gene2id data/gene2ind.txt \
            -cell2id data/cell2ind.txt \
            -drug2id data/drug2ind.txt \
            -genotype data/cell2mutation.txt \
            -fingerprint data/drug2fingerprint.txt \
            -hidden Hidden_sample \
            -result Result_sample \
            -load pretrained_model/drugcell_v1.pt \
            -predict data/rcellminer_test.txt

Traceback (most recent call last):
  File "/Users/yoshitakainoue/code/graph_neural_network_drug_response/DrugCell/code/predict_drugcell_cpu.py", line 80, in <module>
    predict_data, cell2id_mapping, drug2id_mapping = prepare_predict_data(opt.predict, opt.cell2id, opt.drug2id)
  File "/Users/yoshitakainoue/code/graph_neural_network_drug_response/DrugCell/code/util.py", line 105, in prepare_predict_data
    test_feature, test_label = load_train_data(test_file, cell2id_mapping, drug2id_mapping)
  File "/Users/yoshitakainoue/code/graph_neural_network_drug_response/DrugCell/code/util.py", line 93, in load_train_data
    feature.append([cell2id[tokens[0]], drug2id[tokens[1]]])
KeyError: 'CC1=CC(=O)C=CC1=O'


In [5]:
train = pd.read_csv('data/drugcell_train.txt', header=None, sep='\t')
rcellminer = pd.read_csv('data/rcellminer_test.txt', header=None, sep='\t')

In [6]:
train[1]

0                  C1CC(=O)NC(=O)C1N2C(=O)C3=CC=CC=C3C2=O
1       C1CN(CCN1)C(=O)C2=CC=C(C=C2)/C=C/C3=NNC4=CC=CC...
2       COC1=C(C=C2C(=C1)N=CN=C2NC3=CC(=C(C=C3)F)Cl)OC...
3          C1=CC=C2C(=C1)C3=CC=CC=C3N2CCC4=NC5=CC=CC=C5N4
4            CCCCCCCCCCC(C)(C)C(=O)NC1=C(C=C(C=C1OC)OC)OC
                              ...                        
9995    CNC(=O)C1=NC=CC(=C1)OC2=CC=C(C=C2)NC(=O)NC3=CC...
9996    C[C@]12CC[C@@]3(CCC(C[C@@H]3[C@@H]1C(=O)C=C4[C...
9997    CC1=NC=C(C=C1)OC2=C(C=C(C=C2)NC3=NC=NC4=C3C=C(...
9998    CCOC(=O)C1=C(SC2=C1CC(NC2(C)C)(C)C)NC(=O)C3=CC...
9999    C1=CN(C(=O)N=C1N)[C@H]2[C@H]([C@@H]([C@H](O2)C...
Name: 1, Length: 10000, dtype: object

In [7]:
rcellminer[1]

0                                   CC1=CC(=O)C=CC1=O
1                         CCCCCCCCCCCCCCCc1cc(O)ccc1N
2                                   OC(=O)CCCc1ccccc1
3             CCN(CC)CCCNc1c2ccc(Cl)cc2nc3ccc(OC)cc13
4           CCCCCN(CCCCC)CCCNc1c2CCCCc2nc3ccc(Cl)cc13
                             ...                     
981985     NC(=N)NCCCC1N[R]C(=O)C(CC(=O)O)N[R][R]C1=O
981986            OC(=O)CC1N[R][R][R]C(=O)C2CCCN2C1=O
981987       OC(=O)CCC1NC(=O)C(CC(=O)O)N[R][R][R]C1=O
981988          NC(=O)C1N[R][R]C(=O)CSCC2CCCN2[R]C1=O
981989    NC(=O)C1N[R][R][R]C(=O)CSCC2CCCN2[R][R]C1=O
Name: 1, Length: 981990, dtype: object

In [8]:
set(train[1]) & set(rcellminer[1])

{'CCC(=O)OCN1C(=O)C=CC1=O'}

In [8]:
!rm -rf Result_sample && mkdir Result_sample 
!rm -rf Hidden_sample/ && mkdir Hidden_sample

!python code/predict_drugcell_cpu.py \
            -gene2id data/gene2ind.txt \
            -cell2id data/cell2ind.txt \
            -drug2id data/drug2ind.txt \
            -genotype data/cell2mutation.txt \
            -fingerprint data/drug2fingerprint.txt \
            -hidden Hidden_sample \
            -result Result_sample \
            -load pretrained_model/drugcell_v1.pt \
            -predict data/out_.txt

Traceback (most recent call last):
  File "/home/inoue019/code/DrugCell/code/predict_drugcell_cpu.py", line 80, in <module>
    predict_data, cell2id_mapping, drug2id_mapping = prepare_predict_data(opt.predict, opt.cell2id, opt.drug2id)
  File "/home/inoue019/code/DrugCell/code/util.py", line 105, in prepare_predict_data
    test_feature, test_label = load_train_data(test_file, cell2id_mapping, drug2id_mapping)
  File "/home/inoue019/code/DrugCell/code/util.py", line 93, in load_train_data
    feature.append([cell2id[tokens[0]], drug2id[tokens[1]]])
KeyError: 'METHOTREXATE'


In [4]:
!cat data/drugcell_test.txt | head -n 5

LS123_LARGE_INTESTINE	CNC(=O)C1=NC=CC(=C1)OC2=CC(=C(C=C2)NC(=O)NC3=CC(=C(C=C3)Cl)C(F)(F)F)F	0.8942239155087995
RT4_URINARY_TRACT	C[C@@H](C(=O)N[C@H]1C2=CC=CC=C2CCN(C1=O)C)NC(=O)[C@H](C(C)C)O	0.8742612017769524
SCABER_URINARY_TRACT	C1CCN(C1)C(=O)NC2=CC=CC(=C2)NC3=NC=C(C(=N3)NCCC4=CN=CN4)Br	0.9600720748580642
TE617T_SOFT_TISSUE	C1CN(CCC12C(=O)NCN2C3=CC=CC=C3)CCNC(=O)C4=CC5=CC=CC=C5C=C4	0.9873540669080934
MOGGCCM_CENTRAL_NERVOUS_SYSTEM	CC1=CN=C(N=C1NC2=CC(=CC=C2)S(=O)(=O)NC(C)(C)C)NC3=CC=C(C=C3)OCCN4CCCC4	0.8719973751110028
cat: write error: Broken pipe


In [5]:
!cat data/out_.txt | head -n 5

METHOTREXATE	CN(Cc1cnc2nc(N)nc(N)c2n1)c3ccc(cc3)C(=O)N[C@@H](CCC(=O)O)C(=O)O
6-THIOGUANINE	NC1=NC(=S)c2[nH]cnc2N1
6-MERCAPTOPURINE	S=C1N=CNc2nc[nH]c12
Allopurinol	O=C1NC=Nc2[nH]ncc12
Chlorambucil	OC(=O)CCCc1ccc(cc1)N(CCCl)CCCl


In [6]:
os.chdir('/home/inoue019/code/DrugCell/code')

# module for python script

In [7]:
from drugcell_prediction import predict_dcell

In [9]:
!rm -rf Result_sample && mkdir Result_sample 
!rm -rf Hidden_sample/ && mkdir Hidden_sample

predict_dcell(
    '../data/drugcell_test.txt',
    '../data/cell2ind.txt',
    '../data/drug2ind.txt',
    '../data/cell2mutation.txt',
    '../data/drug2fingerprint.txt',
    '../pretrained_model/drugcell_v1.pt',
    'Hidden_sample',
    'Result_sample'
)

Total number of cell lines = 1225
Total number of drugs = 684


/home/inoue019/.local/lib/python3.9/site-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'drugcell_NN.drugcell_nn' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/inoue019/.local/lib/python3.9/site-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/inoue019/.local/lib/python3.9/site-packages/torch/serialization.py:786: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm1d' has changed. you can retrieve the original source code by accessing the object

Test pearson corr	GO:0008150	0.822806
